In [ ]:
%pip install --upgrade --quiet  \
    langchain-pinecone \
    langchain-openai \
    langchain \
    langchain-community \
    pinecone-notebooks\
!pip uninstall -y ragas
!pip install -q pypdf

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\Users\\user\\Downloads\\ten_academy\\week7\\Precision-RAG-For_Enterprise-Grade-RAG-Systems'

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
import time
from langchain_pinecone import PineconeVectorStore


embeddings = OpenAIEmbeddings()

In [6]:
directory_loader = PyPDFLoader("./data/10 Academy Cohort B - Weekly Challenge_ Week - 7.pdf")

In [7]:
raw = directory_loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [9]:
docs = text_splitter.split_documents(raw)
print('split docs', len(docs))

split docs 24


In [10]:
PINECONE_INDEX_NAME = "pdf-doc"
PINECONE_NAME_SPACE = "pdf-namespace"

In [ ]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_api_key

In [12]:
pc = Pinecone(api_key=pinecone_api_key)

In [13]:
index_name = PINECONE_INDEX_NAME  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [32]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

In [14]:
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [16]:
query = "When is the submission deadline for interim report?"
docs_10 = docsearch.similarity_search(query)
print(docs_10[0].page_content)

Deliverables
NOTE:
Document
should
be
a
PDF
stored
in
google
drive
or
published
blog
link.
DO
NOT
SUBMIT
A
LINK
as
PDF!
If
you
want
to
submit
pdf
document,
it
should
be
the
content
of
your
report
not
a
link.
Interim
Submission
-
Wednesday
8pm
UTC
●
Link
to
your
code
in
GitHub
○
Repository
where
you
will
be
using
to
complete
the
tasks
in
this
week's
challenge.
A
minimum
requirement
is
that
you
have
a
well
structured
repository
and
some
coding
progress
is
made.
●
A
review
report
of
your
reading
and
understanding
of
Task
1
and
any
progress
you
made
in
other
tasks.
Feedback
You
may
not
receive
detailed
comments
on
your
interim
submission
but
will
receive
a
grade.
Final
Submission
-
Saturday
8pm
UTC
●
Link
to
your
code
in
GitHub
○
Complete
work
for
Automatic
prompt
generation
○
Complete
work
for
Automatic
evaluation
○
Complete
work
for
Evaluation
Data
Generation
●
A
blog
post
entry
(which
you
can
submit
for
example
to
Medium
publishing)
or
a
pdf
report.
Feedback
You
will
receive


In [17]:
import os
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or ""

chat = ChatOpenAI(
    openai_api_key="",
    model='gpt-3.5-turbo'
)

c:\Users\user\Downloads\ten_academy\week7\Precision-RAG-For_Enterprise-Grade-RAG-Systems\rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant. that generates a good prompt for the given context and \
                  if you do not have context you replay with I do not have context."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [23]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = docsearch.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.
    Contexts:
    {source_knowledge}
    Query: {query}"""
    return augmented_prompt

In [28]:
query = "What are the tasks for the week?"

In [29]:
augment_prompt_txt = augment_prompt(query)

In [ ]:
print(augment_prompt_txt)

In [30]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt_txt
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

The tasks for the week include:
- Understanding the week's challenge
- Understanding prompt engineering
- Understanding Prompt ranking
- Understanding prompt matching
- RAG components
- Techniques to improve R in RAG
- RAG Evaluation Data Generation
- Understanding prompt matching and ranking
- RAG evaluation metrics
- RAGOps - DevOps of RAG development and production deployment

These tasks are part of the training program agenda for the week.
